In [1]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !git clone https://github.com/MJC598/Neuron_Burst_Analysis.git

In [2]:
%matplotlib notebook
import torch.nn as nn
from torch.nn.utils.rnn import pack_sequence
import numpy as np
import torch
torch.manual_seed(0)

from torch.utils.data import TensorDataset, DataLoader
import scipy.io
import random
import pandas as pds
import time

from scipy import stats
from sklearn.metrics import r2_score
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Class Explanations

These are 3 regression RNN-based models. In order to change it to a classifier the 
nn.Linear layers must have their second parameter changed to match the number of 
expected outputs.

* Expected Input Shape: (batch_size, time_sequence, features)

* Input_Size - number of features
* Hidden_Size - number of connections between the hidden layers
* Batch_Size - How many samples you want to push through the network before executing backprop
    (this is a hyperparameter that can change how fast or slow a model converges)
* Batch_First - Should always be set to True to keep input shape the same
* Dropout - Only really does anything with more than 1 layer on the LSTM, RNN, GRU. Useful to help generalize training

In [3]:
class baselineRNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineRNN, self).__init__()
        self.rnn1 = nn.RNN(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        x, h_n  = self.rnn1(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

class baselineLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineLSTM, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
#         self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        self.c0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        x, (h_n, c_n)  = self.rnn(x,(self.h0,self.c0))

        # take all outputs
#         out = self.lin(x[:, :, :])
        out = x

        return out

class baselineGRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineGRU, self).__init__()
        self.rnn = nn.GRU(input_size=input_size,hidden_size=hidden_size,
                          num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        # print(self.h0.shape)
        x, h_n  = self.rnn(x,self.h0)

        # take last cell output
        out = self.lin(x[:, :, :])

        return out
    
class conv1DLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(conv1DLSTM, self).__init__()
        self.c1 = nn.Conv1d(input_size, hidden_size, 3)
        self.p1 = nn.AvgPool1d(3)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 2)
        self.p2 = nn.AvgPool1d(2)
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 1)
#         self.p1 = nn.AvgPool1d(2)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        self.rnn = nn.LSTM(input_size=hidden_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        self.c0 = torch.randn(num_layers, batch_size, hidden_size)
        
    def forward(self, x):
        
        #switch (batch, sequence, feature) to (batch, feature, sequence)
        x = x.transpose(1,2)
        x = self.dropout(x)
        x = self.c1(x)
        x = self.p1(x)
#         x = self.tanh(x)
#         x = self.c2(x)
#         x = self.p2(x)
#         x = self.dropout(x)
        x = self.c3(x)
        x = self.sigmoid(x)
        
        #switch backwards
        x = x.transpose(1,2)
        x, (h_n, c_n)  = self.rnn(x,(self.h0,self.c0))

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out
    
class conv1DGRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(conv1DGRU, self).__init__()
        self.c1 = nn.Conv1d(input_size, hidden_size, 5)
        self.p1 = nn.AvgPool1d(5)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 3)
        self.p2 = nn.AvgPool1d(3)
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 1)
#         self.p1 = nn.AvgPool1d(2)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        self.rnn = nn.GRU(input_size=hidden_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        
    def forward(self, x):
        
        #switch (batch, sequence, feature) to (batch, feature, sequence)
        x = x.transpose(1,2)
        x = self.dropout(x)
        x = self.c1(x)
        x = self.p1(x)
        x = self.c2(x)
#         x = self.p2(x)
#         x = self.c3(x)
        x = self.sigmoid(x)
        
        #switch backwards
        x = x.transpose(1,2)
        x, h_n  = self.rnn(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

In [4]:
TIMESTEPS = 85
FRONT_TIME = -50
BACK_TIME = 40
T_START = 50+FRONT_TIME
T_END = 50+BACK_TIME
MODEL = baselineLSTM
OUTPUT = '5s_lag_FR'
LOSS_FILE = ('losses/bursts/losses_' + str(MODEL) + 
             '_' + OUTPUT + '_' + str(FRONT_TIME) + 
             '_' + str(T_END) + 'full.csv')
PATH = ('models/bursts/' + str(MODEL) + '_' + OUTPUT + 
        '_' + str(FRONT_TIME) + '_' + str(T_END) + 
        '_fullin.pth')
DATA_PATH = 'data/bursts/burst_separatePNITNv2.mat'
COLAB_PRE = 'Neuron_Burst_Analysis/'
if RunningInCOLAB:
    LOSS_FILE = COLAB_PRE + LOSS_FILE
    PATH = COLAB_PRE + PATH
    DATA_PATH = COLAB_PRE + DATA_PATH

# Specific Model Parameters
input_size = 9
hidden_size = 90
output_size = 3
batch_size = 1
num_layers = 1
batch_first = True
dropout = 0.0
epochs = 50

In [5]:
def pad_arr(l, max_dur=0):
    temp = []
    for ar in l:
        npad = ((0,max_dur-ar.shape[0]), (0,0))
        arr = np.pad(ar, pad_width=npad, mode='constant', constant_values=-1)
        temp.append(arr)
    return temp

In [6]:
def get_full_data_from_mat(file_path, output_index=None, type='pre_pn'):
    data = scipy.io.loadmat(file_path)

    full_labels = []
    full_data = []
    
#     print(data['info_collect'][0])
    for i in range(1, data['info_collect'].shape[0]):
        arr = data['info_collect'][i]
        
        pnfr = np.row_stack((arr[2], arr[11]))
        pnaff = np.row_stack((arr[4], arr[13]))
        
        for j in range(pnfr.shape[0]):
            if j >= 2:
                pnfr[j,:] = (pnfr[j-2,:] + pnfr[j-1,:] + pnfr[j,:])/3
                pnaff[j,:] = (pnaff[j-2,:] + pnaff[j-1,:] + pnaff[j,:])/3
            else:
                pnfr[j,:] = pnfr[j,:]
                pnaff[j,:] = pnaff[j,:]
        
#         full_labels.append(np.column_stack((pnfr, 
#                                             clean_data(np.row_stack((arr[3], arr[12])).astype('float')),
#                                             np.row_stack((arr[10], arr[19]))
#                                            ))
#                           )
        
#         full_data.append(np.column_stack((pnfr, 
#                                           clean_data(np.row_stack((arr[3], arr[12]))), 
#                                           pnaff, 
#                                           clean_data(np.row_stack((arr[5], arr[14]))), 
#                                           clean_data(np.row_stack((arr[6], arr[15]))), 
#                                           clean_data(np.row_stack((arr[7], arr[16]))), 
#                                           clean_data(np.row_stack((arr[8], arr[17]))), 
#                                           clean_data(np.row_stack((arr[9], arr[18]))),
#                                           np.row_stack((arr[10], arr[19]))
#                                          ))
#                         )

        full_labels.append(np.column_stack((pnfr, 
                                            np.row_stack((arr[3], arr[12])).astype('float'),
                                            np.row_stack((arr[10], arr[19]))
                                           ))
                          )
        
        full_data.append(np.column_stack((pnfr, 
                                          np.row_stack((arr[3], arr[12])), 
                                          pnaff, 
                                          np.row_stack((arr[5], arr[14])), 
                                          np.row_stack((arr[6], arr[15])), 
                                          np.row_stack((arr[7], arr[16])), 
                                          np.row_stack((arr[8], arr[17])), 
                                          np.row_stack((arr[9], arr[18])),
                                          np.row_stack((arr[10], arr[19]))
                                         ))
                        )

    full_data = np.asarray(full_data,dtype=object)
    full_labels = np.asarray(full_labels,dtype=object)
    
    random.seed(10)
    data_samples = 5472 #5498 
    k = 4352
    
    lag = 1
    front_offset = 11
    training_data = full_data[:k] 
    validation_data = full_data[k:data_samples]
    training_labels = full_labels[:k]
    validation_labels = full_labels[k:data_samples]
    
    td = [] 
    tl = []
    vd = []
    vl = []
    for i, d in enumerate(training_data):
        sample = d[:-1,:]
        label = training_labels[i][1:,:]
        for j in range(sample.shape[0]-previous_time):
            t = sample[j:j+previous_time,:]
            td.append(t.reshape((1,-1)))
            t2 = label[j+previous_time,:]
            tl.append(t2.reshape((1,-1)))
    td = np.vstack(td)
    tl = np.vstack(tl)
    
    
    for i, d in enumerate(validation_data):
        sample = d[:-1,:]
        label = validation_labels[i][1:,:]
        for j in range(sample.shape[0]-previous_time):
            t = sample[j:j+previous_time,:]
            vd.append(t.reshape((1,-1)))
            t2 = label[j+previous_time,:]
            vl.append(t2.reshape((1,-1)))
    vd = np.vstack(vd)
    vl = np.vstack(vl)
    print(vd.shape)
    for i in range(vd.shape[1]):
        if i % 9 == 8:
            td[:,i] = clean_data(td[:,i])
            vd[:,i] = clean_data(vd[:,i])
        if i < (vl.shape[1]-1):
            tl[:,i] = clean_data(tl[:,i])
            vl[:,i] = clean_data(vl[:,i])

    training_dataset = TensorDataset(torch.Tensor(td), torch.Tensor(tl))
    validation_dataset = TensorDataset(torch.Tensor(vd), torch.Tensor(vl))

    return training_dataset, validation_dataset


In [7]:
def get_data_from_mat(file_path, output_index=None, type='pre_pn'):
    data = scipy.io.loadmat(file_path)

    full_labels = []
    full_data = []
    
#     print(data['info_collect'][0])
    for i in range(1, data['info_collect'].shape[0]):
        arr = data['info_collect'][i]
        
        pnfr = np.row_stack((arr[2], arr[11]))
        pnaff = np.row_stack((arr[4], arr[13]))
        
        for j in range(pnfr.shape[0]):
            if j >= 2:
                pnfr[j,:] = (pnfr[j-2,:] + pnfr[j-1,:] + pnfr[j,:])/3
                pnaff[j,:] = (pnaff[j-2,:] + pnaff[j-1,:] + pnaff[j,:])/3
            else:
                pnfr[j,:] = pnfr[j,:]
                pnaff[j,:] = pnaff[j,:]
        
#         full_labels.append(np.column_stack((pnfr, 
#                                             clean_data(np.row_stack((arr[3], arr[12])).astype('float')),
#                                             np.row_stack((arr[10], arr[19]))
#                                            ))
#                           )
        
#         full_data.append(np.column_stack((pnfr, 
#                                           clean_data(np.row_stack((arr[3], arr[12]))), 
#                                           pnaff, 
#                                           clean_data(np.row_stack((arr[5], arr[14]))), 
#                                           clean_data(np.row_stack((arr[6], arr[15]))), 
#                                           clean_data(np.row_stack((arr[7], arr[16]))), 
#                                           clean_data(np.row_stack((arr[8], arr[17]))), 
#                                           clean_data(np.row_stack((arr[9], arr[18]))),
#                                           np.row_stack((arr[10], arr[19]))
#                                          ))
#                         )

        full_labels.append(torch.Tensor(np.column_stack((pnfr, 
                                            np.row_stack((arr[3], arr[12])).astype('float'),
                                            np.row_stack((arr[10], arr[19]))
                                           ))
                                       )
                          )
        
        full_data.append(torch.Tensor(np.column_stack((pnfr, 
                                          np.row_stack((arr[3], arr[12])), 
                                          pnaff, 
                                          np.row_stack((arr[5], arr[14])), 
                                          np.row_stack((arr[6], arr[15])), 
                                          np.row_stack((arr[7], arr[16])), 
                                          np.row_stack((arr[8], arr[17])), 
                                          np.row_stack((arr[9], arr[18])),
                                          np.row_stack((arr[10], arr[19]))
                                         ))
                                     )
                        )
    
    
    random.seed(10)
    data_samples = 5472 #5498 
    k = 4352
    full = np.arange(data_samples)
    training_indices = np.random.choice(full, size=k, replace=False)
    validation_indices = np.delete(full,training_indices)
    
    training_dataset = [full_data[i] for i in training_indices]
    training_labels = [full_labels[i] for i in training_indices]
    
    training_data = pack_sequence(training_dataset,enforce_sorted=False, batch_first=True)
    training_labels = pack_sequence(training_labels,enforce_sorted=False, batch_first=True)
    
    validation_dataset = [full_data[i] for i in validation_indices]
    validation_labels = [full_labels[i] for i in validation_indices]
    
    validation_data = pack_sequence(validation_dataset,enforce_sorted=False, batch_first=True)
    validation_labels = pack_sequence(validation_labels,enforce_sorted=False, batch_first=True)

#     training_dataset = TensorDataset(torch.Tensor(training_data), torch.Tensor(training_labels))
#     validation_dataset = TensorDataset(torch.Tensor(validation_data), torch.Tensor(validation_labels))

    return training_data, training_labels, validation_data, validation_labels
# get_data_from_mat(DATA_PATH)

### Training Method
* Model - Model initialized based on classes above
* Save_Filepath - Where you want to save the model to. Should end with a .pt or .pth extension. This is how you are able to load the model later for testing, etc.
* training_loader - dataloader iterable with training dataset samples
* validation_loader - dataloader iterable with validation dataset samples
* epochs - number of iterations to run

In [8]:
def train_model(model,save_filepath,training_loader,validation_loader,epochs):
    
    epochs_list = []
    train_loss_list = []
    val_loss_list = []
    training_len = len(training_loader[0])
    validation_len = len(validation_loader[0])

    #splitting the dataloaders to generalize code
    data_loaders = {"train": training_loader, "val": validation_loader}

    """
    This is your optimizer. It can be changed but Adam is generally used. 
    Learning rate (alpha in gradient descent) is set to 0.001 but again 
    can easily be adjusted if you are getting issues

    Loss function is set to Mean Squared Error. If you switch to a classifier 
    I'd recommend switching the loss function to nn.CrossEntropyLoss(), but this 
    is also something that can be changed if you feel a better loss function would work
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_func = nn.MSELoss()
#     loss_func = nn.L1Loss()
    decay_rate = 0.93 #decay the lr each step to 93% of previous lr
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

    total_start = time.time()

    """
    You can easily adjust the number of epochs trained here by changing the number in the range
    """
    for epoch in tqdm(range(epochs), position=0, leave=True):
        start = time.time()
        train_loss = 0.0
        val_loss = 0.0
        temp_loss = 100000000000000.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            for phase in data_loaders.keys():  
                x = data_loaders[phase][0]
                y = data_loaders[phase][1]
                output = model(x)     
#                 print(output.size())
#                 print(y.size())
                loss = loss_func(torch.squeeze(output), torch.squeeze(y))  
                #backprop             
                optimizer.zero_grad()           
                if phase == 'train':
                    loss.backward()
                    optimizer.step()                                      

                #calculating total loss
                running_loss += loss.item()
            
            if phase == 'train':
                train_loss = running_loss
                lr_sch.step()
            else:
                val_loss = running_loss

        end = time.time()
        # shows total loss
        if epoch%5 == 0:
            print('[%d, %5d] train loss: %.6f val loss: %.6f' % (epoch + 1, i + 1, train_loss, val_loss))
#         print(end - start)
        
        #saving best model
        if val_loss < temp_loss:
            torch.save(model, save_filepath)
            temp_loss = val_loss
        epochs_list.append(epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
    total_end = time.time()
#     print(total_end - total_start)
    #Creating loss csv
    loss_df = pds.DataFrame(
        {
            'epoch': epochs_list,
            'training loss': train_loss_list,
            'validation loss': val_loss_list
        }
    )
    # Writing loss csv, change path to whatever you want to name it
    lf = ('losses/losses_' + str(MODEL) + '_' 
          + OUTPUT + '_' + str(FRONT_TIME) + '_' 
          + str(T_END) + '_fullin.csv')
    loss_df.to_csv(lf, index=None)
    return train_loss_list, val_loss_list

### R2 Scoring
* Model - same model as sent to train_model
* testing_dataloader - whichever dataloader you want to R2 Score

In [9]:
def r2_score_eval(model, testing_dataloader):
    output_list = []
    labels_list = []
    for i, (x, y) in enumerate(testing_dataloader):
        output = model(x) 
#         print(output.size())
        output_list.append(output.detach().cpu().numpy())
        labels_list.append(y.detach().cpu().numpy())
#     print("Output list size: {}".format(len(output_list)))
#     print(output_list[0].shape)
    output_list = np.concatenate(output_list, axis=0)
    r2output_list = output_list.reshape((-1,output_size))
    labels_list = np.concatenate(labels_list, axis=0)
    r2labels_list = labels_list.reshape((-1,output_size))
#     print(output_list.shape)
#     print(labels_list.shape)
    print(r2_score(r2labels_list, r2output_list))
    return output_list, labels_list

### Program Start

In [10]:
model1 = MODEL(input_size,hidden_size,output_size,batch_size,num_layers,batch_first,dropout)

training_dataset, training_labels, validation_dataset, validation_labels = get_data_from_mat(DATA_PATH)

# Turn datasets into iterable dataloaders
# training_data = DataLoader(dataset=training_dataset,batch_size=batch_size,shuffle=True)
# validation_loader = DataLoader(dataset=validation_dataset,batch_size=batch_size)

p1 = 'models/' + str(MODEL) + '_' + OUTPUT + '_' + str(FRONT_TIME) + '_' + str(T_END) + '_full.pth'
pnfr_training_loss, pnfr_validation_loss = train_model(model1,p1,(training_dataset, training_labels),
                                                       (validation_dataset, validation_labels),epochs)

TypeError: pack_sequence() got an unexpected keyword argument 'batch_first'

In [ ]:
model1 = torch.load(p1)
model1.eval()
t_output_list, t_labels_list = r2_score_eval(model1, training_loader)
v_output_list, v_labels_list = r2_score_eval(model1, validation_loader)
print(t_output_list.shape)
print(t_labels_list.shape)

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2)
fig.tight_layout()
ax[0,0].plot(range(epochs), pnfr_training_loss)
ax[0,0].set_title('Validation Loss')
ax[0,0].set_ylabel('Loss')
ax[0,0].set_xlabel('Epoch')

ax[0,1].plot(range(epochs), pnfr_validation_loss)
ax[0,1].set_title('Training Loss')
ax[0,1].set_ylabel('Loss')
ax[0,1].set_xlabel('Epoch')


ax[1,0].plot(np.arange(v_labels_list.shape[1]), v_labels_list[0,:,0], color='blue')
ax[1,0].plot(np.arange(v_labels_list.shape[1]), v_output_list[0,:,0], color='red')
ax[1,0].set_title('Validation PN FR for Sample 467')
ax[1,0].set_ylabel('PN FR')
ax[1,0].set_xlabel('Time')

ax[1,1].plot(np.arange(t_labels_list.shape[1]), t_labels_list[0,:,0], color='blue')
ax[1,1].plot(np.arange(t_labels_list.shape[1]), t_output_list[0,:,0], color='red')
ax[1,1].set_title('Training PN FR for Sample 467')
ax[1,1].set_ylabel('PN FR')
ax[1,1].set_xlabel('Time')

ax[2,0].plot(np.arange(v_labels_list.shape[1]), v_labels_list[467,:,1], color='blue')
ax[2,0].plot(np.arange(v_labels_list.shape[1]), v_output_list[467,:,1], color='red')
ax[2,0].set_title('Validation ITN FR for Sample 467')
ax[2,0].set_ylabel('ITN FR')
ax[2,0].set_xlabel('Time')

ax[2,1].plot(np.arange(t_labels_list.shape[1]), t_labels_list[467,:,1], color='blue')
ax[2,1].plot(np.arange(t_labels_list.shape[1]), t_output_list[467,:,1], color='red')
ax[2,1].set_title('Training ITN FR for Sample 467')
ax[2,1].set_ylabel('ITN FR')
ax[2,1].set_xlabel('Time')

ax[3,0].plot(np.arange(v_labels_list.shape[1]), v_labels_list[0,:,2], color='blue')
ax[3,0].plot(np.arange(v_labels_list.shape[1]), v_output_list[0,:,2], color='red')
ax[3,0].set_title('Validation LFP for Sample 467')
ax[3,0].set_ylabel('LFP')
ax[3,0].set_xlabel('Time')

ax[3,1].plot(np.arange(t_labels_list.shape[1]), t_labels_list[0,:,2], color='blue')
ax[3,1].plot(np.arange(t_labels_list.shape[1]), t_output_list[0,:,2], color='red')
ax[3,1].set_title('Training LFP for Sample 467')
ax[3,1].set_ylabel('LFP')
ax[3,1].set_xlabel('Time')

plt.show()